In [1]:
import tensorflow as tf
import numpy as np
import scipy
import pandas as pd
import sklearn
from tqdm import tqdm as tqdm
from sklearn.metrics.pairwise import pairwise_kernels
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import division, print_function, unicode_literals
import graph
import threading

%load_ext autoreload
%autoreload 1

C:\Users\nprok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Data preprocessing

In [4]:
meta = pd.read_table("data/d04_text_meta_2016_12_20.txt", index_col=0)
shapes = []
import glob
files = glob.glob("data/raw/4district/*.txt.gz")
df_list = []
for i in tqdm(range(150)):
    df_list.append(pd.read_csv(files[i+111], header=None, usecols=[
                   0, 1, 11], compression='gzip'))
    shapes.append(df_list[-1].shape[0])
df = pd.concat(df_list, copy = False)
df.columns = ['Timestamp', 'Station', 'Avg Speed']

  4%|███▎                                                                              | 6/150 [00:11<04:29,  1.87s/it]

KeyboardInterrupt: 

In [ ]:
total_timesteps = df.shape[0] / df.Station.unique().shape[0]
df = df.groupby('Station').filter(lambda x : x.isnull()['Avg Speed'].sum() < 10 and x.shape[0] == 150*288)

In [ ]:
df = df[df.Station != 400380]

In [ ]:
# df.to_hdf('data/intermidiate/dataset.hdf','w', complevel = 7)

## Preproc

In [ ]:
def get_nearest_meta(lat, long, k, meta):
    meta['distance'] = np.sqrt(
        (meta.Latitude - lat)**2 + (meta.Longitude - long)**2)
    nearest_meta = meta.sort_values('distance').iloc[:k, :]
    return nearest_meta

In [ ]:
# df = pd.read_hdf('data/intermediate/dataset.hdf')

In [ ]:
meta = pd.read_table("data/d04_text_meta_2016_12_20.txt", index_col=0)
stations = df.Station.unique()
meta =meta.loc[stations]

In [ ]:
stations.shape

In [ ]:
meta.plot(kind='scatter', x='Longitude', y='Latitude');
# plt.savefig('detectors.png', dpi = 300)

In [ ]:
lat = 37.40
long = -122

new_meta = get_nearest_meta(lat, long, 256, meta)
df = df[df.Station.isin(new_meta.index.values)]
df.reset_index(drop=True, inplace=True)

In [ ]:
new_meta.plot(kind='scatter', x='Longitude', y='Latitude');
plt.savefig('1.png', dpi = 300)

In [ ]:
new_meta.shape

In [ ]:
locations = new_meta.loc[:, ['Longitude', 'Latitude']].values
adj = graph.compute_graph(locations, gamma=4e3)
plt.imshow(adj)

In [ ]:
total_timesteps = df.groupby('Station').Station.count().max()
data = np.array(np.split(df['Avg Speed'].values, total_timesteps))
X_train, X_valid, X_test,  = np.array_split(data, 3, axis=0)

In [ ]:
from sklearn import preprocessing
std_scaler = preprocessing.StandardScaler()
X_train = std_scaler.fit_transform(X_train.reshape(-1,1)).reshape(-1,new_meta.shape[0])
X_valid = std_scaler.transform(X_valid.reshape(-1,1)).reshape(-1,new_meta.shape[0])
X_test = std_scaler.transform(X_test.reshape(-1,1)).reshape(-1,new_meta.shape[0])
del df

In [ ]:
import pickle
with open('data/processed/std_scaler_256.pkl', 'wb+') as file:
    pickle.dump(std_scaler, file)

In [ ]:
X_train.dump('data/processed/train_256.npy')
X_valid.dump('data/processed/valid_256.npy')
X_test.dump('data/processed/test_256.npy')


In [ ]:
adj.dump('data/processed/adj_256_4e3.npy')

In [ ]:
locations

In [ ]:
adj_norm.sum(axis=0)